# Import Meta Reviewer Suggestions

### Neil D. Lawrence 13th April 2014

Get the meta reviewer names from the google spreadsheet. Then update the reviewer data base, `reviewer.db` with meta reviewers, then export as invites into CMT. Need to make a list of the meta reviewers only from the local data base to upload to CMT for the invites. Potential Area Chairs were stored in a Google doc spreadsheet. For access to the google spreadsheet to work, you need to set, in `cmtutils` the `email` and `password` variables to settings that can access the relevant google doc.

The Google doc contained one entry per meta reviewer in the form of 

```
Name | Institute | Subjects | Email | Answer
```


In [3]:
import cmtutils
from config import config
import pods
# Download our meta-reviewer names from google doc.
spreadsheet_key = config.get('google docs', 'program_committee_key')
spreadsheet_name = '2014-01-19_NIPS_ACs'
res = pods.google.resource(id=spreadsheet_key)
ds = cmtutils.drive_store(res, spreadsheet_name)
ds.read_meta_reviewers()


Now take a look at the suggested area chairs to make sure they were read in correctly.

In [4]:
ds

,Answer,Institute,Name,Subjects,FirstName,MiddleNames,LastName
Email,,,,,,,
jbt@mit.edu,None,MIT,Josh Tenenbaum,1,Josh,,Tenenbaum
jordan@cs.berkeley.edu,N,UC Berkeley,Michael Jordan,1,Michael,,Jordan
sanjivk@google.com,Y,Google,Sanjiv Kumar,1,Sanjiv,,Kumar
mmahoney@cs.stanford.edu,Y,Stanford University,Michael Mahoney,1,Michael,,Mahoney
airoldi@fas.harvard.edu,Y,Harvard University,Edoardo M Airoldi,1,Edoardo,M,Airoldi
dale@cs.ualberta.ca,Y,University of Alberta,Dale Schuurmans,1,Dale,,Schuurmans
lafferty@gmail.com,N,Carnegie Mellon University,John Lafferty,1,John,,Lafferty
dsculley@google.com,Y,Google,D. Sculley,1,D.,,Sculley
claudio.gentile@uninsubria.it,Y,"DiSTA, Universita dell’Insubria, Italy",Claudio Gentile,1,Claudio,,Gentile


To try and keep track of everything we have a local SQLite data base. Here we load that data base in and add invites, labelling them as 'FirstRoundAreaChairInvite'.

In [5]:
# Load in the reviewer data base.
db = cmtutils.reviewerdb('test.db')
fieldname = 'FirstRoundAreaChairInvite'
db.add_field(fieldname) # Add the field if it doesn't exist already

In [6]:
db

,ID,FirstName,MiddleNames,LastName,Institute,Email,ScholarID,Subjects,MSAcademicID,Webpage,...,OtherEmails,IsMetaReviewer,PapersSince2012,NIPSDatabaseNominated2,NipsNominatedCorinnaNeilApproved,FromFullUserList,SecoundRoundInvite,FromAISTATS2012,AreaChairSuggestion,FirstRoundAreaChairInvite
0,1,Aaditya,,Ramdas,CMU,aramdas@cs.cmu.edu,None,None,None,None,...,None,NaN,NaN,NaN,NaN,NaN,None,NaN,None,None
1,2,Aaron,,Courville,University of Montreal,aaron.courville@umontreal.ca,None,None,None,None,...,aaron.courville@gmail.com,NaN,1,NaN,NaN,NaN,None,NaN,None,None
2,3,Aaron,,Hertzmann,Adobe Research,hertzman@dgp.toronto.edu,None,None,None,None,...,hertzman@adobe.com,NaN,1,NaN,NaN,NaN,None,NaN,None,None
3,4,Aarti,,Singh,Carnegie Mellon,aartisingh@cmu.edu,None,None,None,None,...,None,NaN,4,NaN,NaN,NaN,None,NaN,None,None
4,5,Aasa,,Feragen,Max Planck Institutes T�bingen and University ...,aasa@di.ku.dk,None,None,None,None,...,aasa.feragen@tuebingen.mpg.de,NaN,1,NaN,NaN,NaN,None,NaN,None,None
5,6,Abdeslam,,Boularias,Max Planck Institute,boularias@tuebingen.mpg.de,None,None,None,None,...,None,NaN,2,NaN,NaN,NaN,None,NaN,None,None
6,7,Abe,,Friesen,,afriesen@cs.washington.edu,None,None,None,None,...,None,NaN,1,NaN,NaN,NaN,None,NaN,None,None
7,8,Abel,,Rodriguez,UC Santa Cruz,abel@soe.ucsc.edu,None,None,None,None,...,None,NaN,NaN,NaN,NaN,NaN,None,NaN,None,None
8,9,Abhimanyu,,Das,Microsoft Research,abhimand@usc.edu,None,None,None,None,...,abhidas@yahoo-inc.com,NaN,1,NaN,NaN,NaN,None,NaN,None,None
9,10,Abhinav,,Gupta,CMU,abhinavg@cs.cmu.edu,None,None,None,None,...,None,NaN,1,NaN,NaN,NaN,None,NaN,None,None


Now add the users to the data base.

In [ ]:
db.add_users(ds.reviewers, fieldname=fieldname)

We run an SQL command on the data base to count how many invites we have where there is no reply (we assume that the column Metareviewer is set to NULL when there is no reply).

In [7]:
# This is number of non replied first round invitees.
print(db._execute_sql('SELECT COUNT(Active) FROM Reviewers WHERE ' + fieldname + '=1 AND Metareviewer is 0;'))
# This exports reviewers into a file called cmt_export.tsv ready for invite.
#db.export_reviewers(sql='FirstRoundInvite=1')

[(0,)]


## Second Wave of Invites

We proceeded with a second wave of invites on March the 10th, this script loads those invites in, they are in the same spreadsheet file, but on a different sheet.

In [9]:
spreadsheet_name = '2014-03-10_NIPS_ACs'
res = pods.google.resource(id=spreadsheet_key)
ds = cmtutils.drive_store(res, spreadsheet_name)
ds.read_meta_reviewers()
fieldname='SecoundRoundAreaChairInvite'

Once again we can check users have been loaded from the spreadsheet properly.

In [10]:
ds.reviewers

,Answer,Institute,Name,Subjects,FirstName,MiddleNames,LastName
Email,,,,,,,
slivkins@microsoft.com,Y,Microsoft Research,Alex Slivkins,None,Alex,Slivkins,
krausea@ethz.ch,Y,ETH Zurich,Andreas Krause,None,Andreas,,Krause
antonietta.mira@usi.ch,None,University of Lugano,Antonietta Mira,None,Antonietta,,Mira
avrim@cs.cmu.edu,N,CMU,Avrim Blum,None,Avrim,,Blum
bill.triggs@imag.fr,Y,None,Bill Triggs,None,Bill,,Triggs
ckemp@cmu.edu,N,Carnegie Mellon University,Charles Kemp,None,Charles,,Kemp
csutton@inf.ed.ac.uk,Y,University of Edinburgh,Charles Sutton,None,Charles,,Sutton
klein@cs.berkeley.edu,None,UC Berkeley,Dan Klein,None,Dan,,Klein
blei@cs.princeton.edu,Y,Princeton,David Blei,None,David,,Blei


And now we can add them to the data base.

In [ ]:
db.add_users(ds.reviewers, fieldname=fieldname)

We can check how many reviewers we've just added who are not currently meta reviewers with the following SQL command.

In [13]:
print(db._execute_sql("""SELECT COUNT(Active) FROM Reviewers WHERE ' + fieldname + '=1 AND Metareviewer is NULL;"""))
db.export_reviewers(sql=fieldname + '=1') # this exports those reviews into CMT for an upload.

[(0,)]


OperationalError: no such column: SecoundRoundAreaChairInvite

### Counting Number of Metareviewers

This command gives us the total number of meta reviewers by calling SQL on the data base.

In [14]:
db._execute_sql("""SELECT COUNT(Active) FROM Reviewers WHERE Metareviewer=1;""")

[(93,)]